# see the demo notebook here
https://github.com/David-Lor/FastAPI_LightningTalk-Notebook/blob/master/FastAPI.ipynb

In [1]:
# Basic import of FastAPI and function to start running the API with Uvicorn

import uvicorn
from fastapi import FastAPI

app = FastAPI()

def run():
    uvicorn.run(app)

In [2]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!

from multiprocessing import Process
from wait4it import wait_for

_api_process = None

def start_api():
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()
    
    _api_process = Process(target=run, daemon=True)
    _api_process.start()
    wait_for(port=8000)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]

In [3]:
@app.get("/")
def get_root():
    return {"Hello": "World"}

start_api()

INFO:     Started server process [317]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [317]


In [4]:
%%html
<iframe src="http://localhost:8000/docs#/default/get_root__get" width="800" height="600"></iframe>

In [5]:
@app.get("/people/{person_id}")
def get_person(person_id: int):
    return {"person_id": person_id}

start_api()

INFO:     Started server process [338]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:37044 - "GET /people/123abc HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [338]


In [6]:
%%html
<iframe src="http://localhost:8000/docs#/default/get_person_people__person_id__get" width="800" height="600"></iframe>

In [7]:
# Trying to set param as text with letters, when it MUST be int (unparseable)
import requests

r = requests.get("http://localhost:8000/people/123abc")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 422
Response: {'detail': [{'loc': ['path', 'person_id'], 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}]}


In [8]:
from typing import Optional

@app.get("/people")
def list_people(surname: str, city: Optional[str] = None, limit: int = 20):
    return {
        "filters": {
            "surname": surname,
            "city": city
        },
        "limit": limit
    }

start_api()

INFO:     Started server process [361]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:43578 - "GET /people?surname=foolanito HTTP/1.1" 200 OK
INFO:     127.0.0.1:43720 - "GET /people HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [361]


In [10]:
%%html
<iframe src="http://localhost:8000/docs#/default/list_people_people_get" width="800" height="600"></iframe>

In [11]:
# Get with "surname" param only
import requests

r = requests.get("http://localhost:8000/people?surname=foolanito")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 200
Response: {'filters': {'surname': 'foolanito', 'city': None}, 'limit': 20}


In [12]:
# Get without params ("surname" is required)
import requests

r = requests.get("http://localhost:8000/people")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 422
Response: {'detail': [{'loc': ['query', 'surname'], 'msg': 'field required', 'type': 'value_error.missing'}]}


In [13]:
from pydantic import BaseModel
from typing import Optional

class Person(BaseModel):
    name: str
    age: Optional[int]
    profession: str = "Student"

@app.post("/people")
def create_person(person: Person):
    return person.dict()

start_api()

INFO:     Started server process [387]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [14]:
%%html
<iframe src="http://localhost:8000/docs#/default/create_person_people_post" width="800" height="600"></iframe>